In [1]:
import json
import pandas as pd
import os
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import shape, Polygon, MultiPolygon
from shapely.wkt import loads
import requests

In [ ]:
print(os.getpid())

In [5]:
df = pd.read_csv("/mntssd/mnt3/shanshanbai/nlpinearthobservation/synthetic_data/merged.csv", sep=",")

In [6]:
df['geometry'] = df['geometry'].apply(loads)

In [ ]:
df.geometry[0]

In [23]:
# Define Overpass API endpoint
url = "https://overpass-api.de/api/interpreter"

In [47]:
bounds = df.geometry[5].bounds  # (minx, miny, maxx, maxy)
south, west, north, east = bounds[1], bounds[0], bounds[3], bounds[2]

    # Create Overpass query using the bounding box
query = f"""[out:json];
(
  way["building"]({bounds[1]},{bounds[0]},{bounds[3]},{bounds[2]});
  relation["building"]({bounds[1]},{bounds[0]},{bounds[3]},{bounds[2]});
  node["building"]({bounds[1]},{bounds[0]},{bounds[3]},{bounds[2]});
);
out body;
>;
out skel qt;
"""
response = requests.get(url, params={"data": query})
response.raise_for_status()
data = response.json()
data

{'version': 0.6,
 'generator': 'Overpass API 0.7.62.4 2390de5a',
 'osm3s': {'timestamp_osm_base': '2024-12-27T21:47:45Z',
  'copyright': 'The data included in this document is from www.openstreetmap.org. The data is made available under ODbL.'},
 'elements': [{'type': 'way',
   'id': 11286296,
   'nodes': [99869020,
    11115784747,
    1967214790,
    100261869,
    100251360,
    100251361,
    99869010,
    11115784746,
    99869014,
    99869015,
    99869016,
    99869017,
    99869020],
   'tags': {'building': 'roof',
    'building:colour': 'white',
    'building:part': 'yes',
    'height': '23.5',
    'layer': '1',
    'min_height': '23',
    'roof:shape': 'flat',
    'start_date': '2003-12-10'}},
  {'type': 'way',
   'id': 11342979,
   'nodes': [1967214788,
    9269743906,
    1967214792,
    9269743905,
    100898140,
    9269743903,
    100898139,
    9269743873,
    1967214788],
   'tags': {'building': 'steps',
    'building:colour': 'white',
    'height': '5',
    'roof:dir

In [40]:
# Initialize the result dictionary
result = {
    'building tags': [],
    'building names': [],
    'building operators': []
}

# Iterate over elements in the data
for element in data["elements"]:
    if element["type"] in ["way", "relation"]:  # Process only 'way' and 'relation'
        tags = element.get("tags", {})
        
        # Extract building tag
        building = tags.get("building")
        if building and building != "yes" and building not in result["building tags"]:
            result["building tags"].append(building)
        
        # Extract building name
        name = tags.get("name")
        if name and name not in result["building names"]:
            result["building names"].append(name)
        
        # Extract operator
        operator = tags.get("operator")
        if operator and operator not in result["building operators"]:
            result["building operators"].append(operator)


In [41]:
result

{'building tags': ['hospital', 'university'],
 'building names': ['Michael Rutter Centre',
  'Felix Post Unit',
  'Henry Wellcome Building for Psychology',
  'Institute of Psychiatry, Psychology & Neuroscience'],
 'building operators': ['South London and Maudsley NHS Foundation Trust',
  "King's College London"]}